In [1]:
pip install dash plotly yfinance pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from dash import Dash, dcc, html, Input, Output

# Initialize app
app = Dash(__name__)
app.title = "📈 Stock Market Visualizer"

# List of example tickers
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'INFY', 'TCS.NS']

# App layout
app.layout = html.Div(style={'backgroundColor': '#111111', 'padding': '20px'}, children=[
    html.H1("📊 Stock Market Visualizer", style={'color': '#00F0FF', 'textAlign': 'center'}),
    
    html.Div([
        html.Label("Select Stock:", style={'color': 'white'}),
        dcc.Dropdown(
            id='ticker-dropdown',
            options=[{'label': t, 'value': t} for t in tickers],
            value='AAPL',
            style={'width': '300px'}
        ),
        
        html.Br(),
        
        html.Label("Select Date Range:", style={'color': 'white'}),
        dcc.DatePickerRange(
            id='date-range',
            start_date=pd.to_datetime('2023-01-01'),
            end_date=pd.to_datetime('2024-12-31'),
            style={'color': '#000'}
        ),
    ]),
    
    html.Br(),
    
    dcc.Graph(id='line-chart'),
    dcc.Graph(id='candlestick-chart')
])

# Callbacks
@app.callback(
    [Output('line-chart', 'figure'),
     Output('candlestick-chart', 'figure')],
    [Input('ticker-dropdown', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_graphs(ticker, start_date, end_date):
    start = pd.to_datetime(start_date).date()
    end = pd.to_datetime(end_date).date()
    df = yf.download(ticker, start=start, end=end, auto_adjust=False)


    
    # Calculate moving averages
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    
    # Line chart with MA
    line_fig = go.Figure()
    line_fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price', line=dict(color='#00F0FF')))
    line_fig.add_trace(go.Scatter(x=df.index, y=df['MA20'], mode='lines', name='MA 20', line=dict(color='orange', dash='dot')))
    line_fig.add_trace(go.Scatter(x=df.index, y=df['MA50'], mode='lines', name='MA 50', line=dict(color='red', dash='dash')))

    line_fig.update_layout(
        title=f'{ticker} - Close Price with Moving Averages',
        paper_bgcolor='#111111',
        plot_bgcolor='#111111',
        font_color='white'
    )

    # Candlestick chart
    candle_fig = go.Figure(data=[go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        increasing_line_color='green',
        decreasing_line_color='red'
    )])
    
    candle_fig.update_layout(
        title=f'{ticker} - Candlestick Chart',
        xaxis_title='Date',
        yaxis_title='Price',
        paper_bgcolor='#111111',
        plot_bgcolor='#111111',
        font_color='white'
    )
    
    return line_fig, candle_fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)